**Purpose:** Notes on good ideas for organizing and developing a Python project.

## Project layout

Use a [`src`-layout](https://setuptools.pypa.io/en/latest/userguide/package_discovery.html#src-layout) package layout.

## Virtual environment

Use [`venv` for virtual environment](https://docs.python.org/3/library/venv.html).

`conda` can be used but are best for scientific computing.

## Editable install

Install the package in [development mode / editable mode](https://setuptools.readthedocs.io/en/latest/userguide/quickstart.html#development-mode). This enables us to edit the package on the fly without the need to reinstall it.

## Jupyter magic

Use the [`autoreload` magic command when making prototypes in a notebook](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html). And use [Cython magic commands](https://ipython.org/ipython-doc/2/config/extensions/cythonmagic.html) when prototyping Cython code.

## Testing

Use `pytest` and put the tests in a [`tests/` folder in the same directory](https://docs.pytest.org/en/7.1.x/explanation/goodpractices.html#tests-outside-application-code). 

Often used tools for testing in Python are `pytest` (most common and the best) and `unittest`.

## Documentation

Use `sphinx` for documentation. `mkdocs` seems like the new kid on the block but `sphinx` is older and more used.

Write doctests when writing documentation. [Doctest can be run with pytest](https://docs.pytest.org/en/7.1.x/how-to/doctest.html).

## Type Checking

Use `pyright` for type checking.

## Code style formatter

Often used are `flake8` or for automatic code formatting people often use `black`.

## Pre-commit hooks

[pre-commit](https://pre-commit.com) is often used to run a series of checks before making commits to git e.g. `black` code formatting, `pyright`, `isort` etc.

## Logging

The `logging` module is a nice debugging tool in Python, use it.

## Cookiecutter

[cookiecutter](https://github.com/cookiecutter/cookiecutter) can be used to create a template Python project.

## PEP 8

Use [PEP 8](https://peps.python.org/pep-0008). And think about the [public and non-public API](https://peps.python.org/pep-0008/#public-and-internal-interfaces). Default to non-public attributes, API etc.

## Branching Model

Use Github flow. [See this discussion](https://medium.com/@sreekanth.thummala/choosing-the-right-git-branching-strategy-a-comparative-analysis-f5e635443423).

## CI/CD Pipelines

Use Githubs workflows for CI/CD pipelines.

## Publish Package

Use the pypi package index.

You can create your own package index on premise (`conda` is pretty easy). Other solutions for an on premise package index exists like [fury](https://gemfury.com).

## Versioning

Use [semantic versioning](https://semver.org) e.g. MAJOR.MINOR.PATCH and `git tag` each version and push to the remote.

[`versioneer`](https://github.com/python-versioneer/python-versioneer) seems like a cool project to automate parts of the versioning process.